In [1]:
#!pip install unidecode
#!pip install pprint
#!pip install pandas
from pprint import pprint
import json
from unidecode import unidecode
import html
import re
import os
import csv
import pandas as pd
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
n_instances = 1000
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
print(len(subj_docs), len(obj_docs))

1000 1000


In [5]:
train_subj_docs = subj_docs[:800]
test_subj_docs = subj_docs[800:1000]
train_obj_docs = obj_docs[:800]
test_obj_docs = obj_docs[800:1000]
training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])

In [6]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

In [7]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)

In [8]:
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.835
F-measure [obj]: 0.8263157894736842
F-measure [subj]: 0.8428571428571429
Precision [obj]: 0.8722222222222222
Precision [subj]: 0.8045454545454546
Recall [obj]: 0.785
Recall [subj]: 0.885


In [20]:
TagRemove = re.compile(r'<[^>]+>')


def RemoveTags(text):
    return TagRemove.sub("", text)

def CleanText(text):
    return unidecode(RemoveTags(html.unescape(text.strip())))

In [16]:
sid = SentimentIntensityAnalyzer()

In [21]:
LABELSFILE = "Text.csv"
TEXTFILE = "AllText.json"
FileJson = json.load(open(TEXTFILE))

Labels = {}
Text = {}

with open(LABELSFILE, 'r') as f:
    File = csv.DictReader(f, delimiter=',')
    for row in File:
        ID = int(row['ID'])        
        if row['ClearMajority'] != '-':
            Labels[ID] = (row['ClearMajority'])
        elif row['SoftMajority'] != '-':
            Labels[ID] = int(row['SoftMajority'])
        elif row['damon'] != '-':
            Labels[ID] = int(row['damon'])

for ID in Labels.keys():
    Text[ID] = FileJson[str(ID)]
    print(sid.polarity_scores(Text[ID]), Labels[ID])


{'neg': 0.29, 'neu': 0.57, 'pos': 0.14, 'compound': -0.4404} 2
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 1
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 0
{'neg': 0.0, 'neu': 0.787, 'pos': 0.213, 'compound': 0.69} 1
{'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'compound': 0.4215} -1
{'neg': 0.0, 'neu': 0.726, 'pos': 0.274, 'compound': 0.7506} 1
{'neg': 0.0, 'neu': 0.668, 'pos': 0.332, 'compound': 0.54} 1
{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'compound': 0.3612} 0
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 0
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 1
{'neg': 0.177, 'neu': 0.686, 'pos': 0.137, 'compound': -0.34} -2
{'neg': 0.0, 'neu': 0.514, 'pos': 0.486, 'compound': 0.7667} 1
{'neg': 0.0, 'neu': 0.838, 'pos': 0.162, 'compound': 0.4019} -1
{'neg': 0.029, 'neu': 0.883, 'pos': 0.088, 'compound': 0.7676} -1
{'neg': 0.038, 'neu': 0.907, 'pos': 0.054, 'compound': 0.0026} -2
{'neg': 0.0, 'neu': 0.86, 'pos': 0.14, 'compound': 0.7081} 1
{'neg': 0.0, 'ne

{'neg': 0.0, 'neu': 0.822, 'pos': 0.178, 'compound': 0.7906} 0
{'neg': 0.278, 'neu': 0.686, 'pos': 0.035, 'compound': -0.9081} -2
{'neg': 0.189, 'neu': 0.552, 'pos': 0.259, 'compound': 0.2225} 1
{'neg': 0.0, 'neu': 0.533, 'pos': 0.467, 'compound': 0.9169} 1
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 1
{'neg': 0.0, 'neu': 0.828, 'pos': 0.172, 'compound': 0.577} 1
{'neg': 0.098, 'neu': 0.446, 'pos': 0.455, 'compound': 0.7906} -1
{'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'compound': 0.4019} 1
{'neg': 0.128, 'neu': 0.561, 'pos': 0.311, 'compound': 0.5574} -2
{'neg': 0.254, 'neu': 0.493, 'pos': 0.254, 'compound': 0.0} -1
{'neg': 0.04, 'neu': 0.877, 'pos': 0.084, 'compound': 0.2732} -1
{'neg': 0.126, 'neu': 0.682, 'pos': 0.192, 'compound': 0.7472} 1
{'neg': 0.05, 'neu': 0.822, 'pos': 0.127, 'compound': 0.6204} 1
{'neg': 0.04, 'neu': 0.96, 'pos': 0.0, 'compound': -0.3182} 1
{'neg': 0.0, 'neu': 0.72, 'pos': 0.28, 'compound': 0.8271} 2
{'neg': 0.0, 'neu': 0.803, 'pos': 0.197, 'compoun

{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'compound': 0.4753} -1
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} -2
{'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'compound': 0.4981} 2
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 0
{'neg': 0.0, 'neu': 0.767, 'pos': 0.233, 'compound': 0.6908} 1
{'neg': 0.0, 'neu': 0.815, 'pos': 0.185, 'compound': 0.5106} -1
{'neg': 0.0, 'neu': 0.819, 'pos': 0.181, 'compound': 0.8225} 2
{'neg': 0.178, 'neu': 0.533, 'pos': 0.29, 'compound': 0.2263} -1
{'neg': 0.0, 'neu': 0.681, 'pos': 0.319, 'compound': 0.9701} 1
{'neg': 0.0, 'neu': 0.748, 'pos': 0.252, 'compound': 0.8786} 1
{'neg': 0.053, 'neu': 0.791, 'pos': 0.156, 'compound': 0.8234} 1
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 0
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} -2
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 0
{'neg': 0.117, 'neu': 0.68, 'pos': 0.203, 'compound': 0.3111} -1
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} 2
{'neg': 0.027, 'neu': 0.